In [1]:
import pandas as pd

In [2]:
list1 = pd.read_csv('List1.csv')
list2 = pd.read_csv('List2.csv')

In [3]:
list1 = list1.rename(columns={'INSTNM': 'instnm', 'STABBR': 'state'})
list2 = list2.rename(columns={'Institution Name': 'instnm', 'State': 'state'})

In [4]:
list1['instnm'] = list1['instnm'].str.lower()
list1['state'] = list1['state'].str.lower()

list2['instnm'] = list2['instnm'].str.lower()
list2['state'] = list2['state'].str.lower()

In [5]:
matches = pd.merge(list1, list2, on=['instnm', 'state'], how='inner')
matches.to_csv('matched.csv', index=False)

In [11]:
uniqu = matches.nunique()

In [12]:
uniqu

ID        1971
instnm    1949
CITY      1123
state       58
dtype: int64

In [22]:
matches = matches.drop_duplicates()

In [21]:
matches['instnm'].duplicated().sum()

23

In [19]:
matches.duplicated().sum()

1

In [18]:
matches.shape

(1972, 4)

In [6]:
unmatched = pd.merge(list2, list1, on=['instnm', 'state'], how='left', indicator=True)
unmatched = unmatched[unmatched['_merge'] == 'left_only'].drop(columns=['_merge'])
unmatched_count = unmatched.shape[0]
print(f"Number of unmatched records: {unmatched_count}")

Number of unmatched records: 73


In [7]:
unmatched.to_csv('unmatched.csv', index=False)

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('fuzzy_matches.csv')
df1_institutions = df['df1_inst'].tolist()
df2_institutions = df['df2_inst'].tolist()

In [4]:
from googlesearch import search
import time


In [5]:
def get_college_address_url(institution_name):
    query = f"{institution_name} address"
    search_results = search(query, num=1, stop=1)
    for url in search_results:
        return url
    return None

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
def scrape_address_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Adjust the tag and class based on the structure of the web page
        address = soup.find('p').get_text()
        return address.strip()
    except Exception as e:
        return "Address not found"

In [8]:
df1_addresses = []
df2_addresses = []

In [10]:
for institution in df1_institutions:
    try:
        url = get_college_address_url(institution)
        if url:
            address = scrape_address_from_url(url)
        else:
            address = "Address not found"
        print(f"Processed df1_inst: {institution} - {address}")
    except Exception as e:
        address = "Address not found"
    df1_addresses.append(address)
      # Adding delay to avoid rate limiting

# Process df2_inst column
for institution in df2_institutions:
    try:
        url = get_college_address_url(institution)
        if url:
            address = scrape_address_from_url(url)
        else:
            address = "Address not found"
        print(f"Processed df2_inst: {institution} - {address}")
    except Exception as e:
        address = "Address not found"
    df2_addresses.append(address)
      # Adding delay to avoid rate limiting

# Add the addresses to the DataFrame and save to a new CSV
df['df1_address'] = df1_addresses
df['df2_address'] = df2_addresses
df.to_csv('colleges_with_addresses.csv', index=False)


ValueError: Length of values (2732) does not match length of index (1978)